# Facial Recognition Using Convolutional Neural Networks 

## Imports

In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pickle
%matplotlib inline

## Data Pre-Processing & Augmentation

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2, # For randomly applying shearing transformations
                                  zoom_range=0.2, # For randomly zooming inside pictures
                                  horizontal_flip=True, # For randomly flipping half of the images horizontally
                                  rotation_range=40, # A value in degrees (0-180), a range within which to randomly rotate pictures
                                  width_shift_range=0.2, # For randdomly rotating pictures
                                  height_shift_range=0.2, 
                                  fill_mode='nearest' # The strategy used for filling in newly created pixels after a transformation
                                 )


validation_generator = test_datagen.flow_from_directory("D:/RT4/PFA/dataset/test", batch_size=1, 
                                                        class_mode='categorical', 
                                                       )


## Convolutional Neural Network

In [ ]:
class Image_Classifier():
    def __init__(self, train_folder="dataset/train", validation_folder="dataset/test", 
                  inshape=(50,50,3), num_classes=2, num_train=21, num_validation=8):
        self.train_folder = train_folder
        self.validation_folder =  validation_folder
        self.inshape = inshape
        self.num_classes = num_classes
        self.num_train = num_train
        self.num_validation = num_validation
        
    def model(self, learning_rate=0.001):
        model = Sequential()

        model.add(Convolution2D(filters=32,
                                kernel_size=(3, 3),
                                strides=(1, 1), 
                                input_shape=self.inshape, 
                                data_format="channels_last")
                 )
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))

        model.add(Convolution2D(filters = 16,
                                kernel_size=(3, 3),
                                strides=(1, 1)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.3))
        
        model.add(Flatten())

        model.add(Dense(200))
        model.add(Activation('relu'))
        model.add(Dropout(0.3))

        model.add(Dense(self.num_classes, activation='softmax'))

        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        return model
    
    def train(self, batch_size=128, epochs=12, learning_rate=0.001, model_file="my_model.h5", refine = False):
        
        # The augmentation configuration used for training
        train_datagen = ImageDataGenerator(rescale=1./255, 
                                            shear_range=0.2, 
                                            zoom_range=0.2, 
                                            horizontal_flip=True, 
                                            rotation_range=40,  
                                            width_shift_range=0.2,
                                            height_shift_range=0.2, 
                                            fill_mode='nearest' 
                                          )

        # The augmentation configuration used for testing (rescale only)
        test_datagen = ImageDataGenerator(rescale=1./255)

        # A generator that will read pictures found in subfolers of 'data/train' 
        # and indefinitely generate augmented image data
        train_generator = train_datagen.flow_from_directory(self.train_folder,  
                                                            target_size=(50,50),  
                                                            batch_size=batch_size, 
                                                           )
        # Similar generator for validation data
        validation_generator = test_datagen.flow_from_directory(self.validation_folder, 
                                                                target_size=(50,50),
                                                                batch_size=batch_size, 
                                                               )
        
        # Create labels file
        dict_label = train_generator.class_indices
        dict_label = {v: k for k, v in dict_label.items()}
        with open('labels.pkl', 'wb') as f:
            pickle.dump(dict_label, f)
        
        if refine:
            model = load_model(model_file)
        else:
            model = self.model(learning_rate=learning_rate)
        
        model.fit_generator(train_generator,
                            steps_per_epoch=self.num_train // batch_size,
                            epochs=epochs,
                            validation_data=validation_generator,
                            validation_steps=self.num_validation // batch_size)
        
        # Saving the model
        model.save(model_file)
        
        
    def predict(self, x, model_file="my_model.h5"):
        model = load_model(model_file)
        prediction = model.predict(x)
        return prediction

## Training 

In [ ]:
H_DIM = 300
W_DIM = 300

im_classifier = Image_Classifier(train_folder ="dataset/train",
                                 validation_folder = "dataset/test", 
                                 inshape=(H_DIM,W_DIM,3),
                                 num_classes=2,
                                 num_train=1000,
                                 num_validation=200)

im_classifier.train(batch_size=2,
                    epochs=20,
                    learning_rate=0.01)

## Prediction

In [ ]:
im = Image.open("dataset/test/Skander_Meghirbi/SkanderMeghirbi21.JPG")
im = im.resize((H_DIM,W_DIM), Image.ANTIALIAS)
im = np.array(im)
im = np.expand_dims(im, axis=0)

im_classifier = Image_Classifier()

prediction = im_classifier.predict(im, model_file="my_model.h5")
pred_idx = prediction[0].argmax()

with open('labels.pkl', 'rb') as f:
            dict_label = pickle.load(f)

print("The corresponding person is ", dict_label[pred_idx], "with probability of %d", prediction[0][pred_idx]*100)